2. Funcion de onda Gaussiana

In [43]:
#Importing the necessary modules for the code
include("QuantumOsc.jl") #Use the module in the same folder, it already compiles it. The Moduel QuantumOsc should be in the same folder, else specify directory

using .QuantumOsc
using Plots
using LaTeXStrings
using QuadGK
using Statistics

ħ = 1
m = 1
ω = 1
s = sqrt(2*ħ / (m*ω))

QuantumOsc.set_constants(ħ, m, ω) #This is a command for settin constants ħ, m, ω respectively in the functions used in QuantumOsc Module

1.4142135623730951

In [44]:
#Variables for simulation (Would be nive to just compile them once)
#Spacial Variable x
stpx = 0.01
lim = 7
x = Float64.(-lim:stpx:lim)

#Temporal Variable t
stpt = 0.01
periods = 2
tf = periods*(2π/ω)
t = Float64.(0:stpt:tf)

1257-element Vector{Float64}:
  0.0
  0.01
  0.02
  0.03
  0.04
  0.05
  0.06
  0.07
  0.08
  0.09
  0.1
  0.11
  0.12
  ⋮
 12.450000000000001
 12.46
 12.47
 12.48
 12.49
 12.5
 12.51
 12.52
 12.530000000000001
 12.540000000000001
 12.55
 12.56

In [45]:
#State 2
#Constants of the problem (And the state function f(x))
id = "var" #Case number/identifier
if id == "b1"
    b = 0.7s
    c = 1.5s
elseif id == "b2"
    b = 1s
    c = 1.5s
elseif id == "b3"
    b = 1.5s
    c = 1.5s
elseif id == "var" #Variation for playing with the initial values
    b = 2s
    c = 1s
end

A_2 = (2/(b^2*π))^(1/4) #Normalization value, analytically calculated
ψ_2(x) = A_2.*exp.(-(x.-c).^2 ./ b^2)

num_s = 22 #Number of eigenstates with which we will reconstruct the function

22

In [46]:
#Check integral and plot the function (same as state 1)
fun(x) = abs(ψ_2(x))^2 #Dummy fun, for |ψ_1|^2 to be callable as a function
integ = quadgk(fun, -lim, lim)[1] #Now we use gaussian quadrature
println(integ)

plot(x, abs.(ψ_2(x)).^2, title = L"Probability Distribution $|\psi_2|^2$ at $t=0$ (%$id)", label = L"$|\psi_2|^2$")
savefig("ex2mp\\state2t0_$id.png")

0.9999608818126335


"c:\\Users\\emino\\OneDrive\\Escritorio\\Tareas\\5to Semestre\\Cuantica\\Cuantica MP1 QuantumOsc Final\\ex2mp\\state2t0_var.png"

In [47]:
#Calculation of Constants
#Numerically
function num_cn(n)
    fun(x) = ψ_2(x) .* φ_n(x,n) #Dummy fun
    #numcn = quadgk(fun, -10, 10)[1]
    numcn = integral(fun, -lim, lim)
return numcn
end

#Analytically
function ana_cn(n)
    if id == "b2"
        α_n = A_2 * (2/(pi*s^2))^(1/4) * sqrt(2^n * factorial(n))^(-1) * exp(-c^2/b^2)
        α_n = Float64(α_n)
        β = (s^(-2) + b^(-2))
        int_n =  exp(c^2/(b^4 * β)) * sqrt(pi/β) * (c/(β*b^2))^n * 2^n
        
        r = α_n * int_n
    else
        #Constants to reduce the length of the expression
        α_n = A_2 * (2/(pi*s^2))^(1/4) * sqrt(2^big(n) * factorial(big(n)))^(-1)
        α_n = Float64(α_n)
        β = (s^(-2) + b^(-2)) #Constant that will appear in the integral (p)
        com_γ = Complex(1 - 2 / (s^2 * β))

        #Value of the integral involving hermite and gaussian with x term
        int_n =  exp(c^2/(b^4 * β)) * sqrt(pi/β) * com_γ^(n/2) * complex_hermiteN(n, (sqrt(2) * c  / ((s*b^2)*β*sqrt(com_γ)))) #Using expression of Belafhal et al. we are integrating the hermite times the gaussian with the component e^2qx
        
        r = α_n * int_n * exp(-c^2/b^2)
    end
return r
end

ana_cn (generic function with 1 method)

In [48]:
#Comparison of the constants for different eigenstates, just checking our expressions
nf = 18
nerror = zeros(nf+1)
for ni ∈ 0:nf
    ncni = num_cn(ni)
    acni = ana_cn(ni)

    println("Numerical constant ", ni, ": ", round(ncni; digits=4))
    println("Analytic constant ", ni, ": ", round(acni; digits=4))
    
    #Checking for the diference:
    nerror[ni+1] = round(abs(ncni - acni); digits=4)
end

println("Mean Error:", mean(nerror))

Numerical constant 0: 0.7323
Analytic constant 0: 0.7323 + 0.0im
Numerical constant 1: 0.2929
Analytic constant 1: 0.2929 + 0.0im
Numerical constant 2: 0.3935
Analytic constant 2: 0.3935 + 0.0im
Numerical constant 3: 0.2344
Analytic constant 3: 0.2344 + 0.0im
Numerical constant 4: 0.2514
Analytic constant 4: 0.2514 + 0.0im
Numerical constant 5: 0.1707
Analytic constant 5: 0.1707 + 0.0im
Numerical constant 6: 0.1656
Analytic constant 6: 0.1656 + 0.0im
Numerical constant 7: 0.1199
Analytic constant 7: 0.1199 + 0.0im
Numerical constant 8: 0.1099
Analytic constant 8: 0.1099 + 0.0im
Numerical constant 9: 0.0825
Analytic constant 9: 0.0825 + 0.0im
Numerical constant 10: 0.073
Analytic constant 10: 0.073 + 0.0im
Numerical constant 11: 0.056
Analytic constant 11: 0.056 + 0.0im
Numerical constant 12: 0.0484
Analytic constant 12: 0.0484 + 0.0im
Numerical constant 13: 0.0376
Analytic constant 13: 0.0376 + 0.0im
Numerical constant 14: 0.032
Analytic constant 14: 0.032 + 0.0im
Numerical constant 15

In [49]:
#With the given eigenstates considered, construct the function
nsv = zeros(num_s)

for ni ∈ 0:num_s-1
    nsv[ni+1] = ana_cn(ni)
end

In [50]:
#Making the function with the superposition of the states
#Making a function was the only way I found to make the new function as a for of the diferent states num_s

function ψt_2(x,t) #The reconstruction of the function
    ψt_2 = 0 
    for i ∈ 1:num_s #Necessary to recreate the function using num_s states
        ψt_2 = ψt_2 .+ nsv[i].*φt_n(x,t,(i-1))
    end
    return ψt_2
end

plot(x, abs.(ψt_2(x, 0)).^2, title=L"$\vert\psi_2\vert^2$ at time $t=0$ (%$id)", label = L"$|\psi_2|^2$")
savefig("ex2mp\\state2t0_reconstructed_$id.png")

"c:\\Users\\emino\\OneDrive\\Escritorio\\Tareas\\5to Semestre\\Cuantica\\Cuantica MP1 QuantumOsc Final\\ex2mp\\state2t0_reconstructed_var.png"

In [51]:
#Heatmap of the function through time and the expected value
heatmap_matrix, exp_x_val = heatmap_gen(ψt_2, x, t)

([1.9288215929078386e-7 2.0340605818310927e-7 … 9.435485830091645e-6 8.813535918632707e-6; 1.929101117276635e-7 2.034359799167471e-7 … 9.431510645368573e-6 8.809845669236076e-6; … ; 1.9295702946226153e-7 2.0348620270338208e-7 … 9.424837350963675e-6 8.80365068845344e-6; 1.9289350593654086e-7 2.0341820423986427e-7 … 9.433872249801485e-6 8.812037998352074e-6], [1.4133485085690538, 1.4132778445771406, 1.4130658596579428, 1.4127125749804352, 1.4122180258240884, 1.4115822615756222, 1.4108053457244678, 1.4098873558569276, 1.4088283836490259, 1.407628534858051  …  1.406790751097592, 1.4080802853288867, 1.4092290176893094, 1.4102368334559543, 1.4111036319811059, 1.4118293267015363, 1.4124138451464834, 1.4128571289443246, 1.41315913382795, 1.4133198296388279])

In [52]:
#Plotting the heatmap with the expected value
heatmap(x, t, heatmap_matrix, xlabel="x", ylabel="t", title=L"Heatmap of $|\psi_2|^2$ (%$id)")
plot!( exp_x_val, t, label=L"$\langle x \rangle$")
savefig("ex2mp\\heatmap_exp_val_st2_$id.png")

"c:\\Users\\emino\\OneDrive\\Escritorio\\Tareas\\5to Semestre\\Cuantica\\Cuantica MP1 QuantumOsc Final\\ex2mp\\heatmap_exp_val_st2_var.png"

In [53]:
#Evolution of the function through time. Animation
anim = @animate for i ∈ 1:length(t)
    plot(x, heatmap_matrix[i, :], title = L"Evolution of $\vert\psi_2\vert^2$ over time $T=\frac{4\pi}{\omega}$ (%$id)", ylims=(-0.05 , 1), label=false)
end

# Save animation
gif(anim, "ex2mp\\state2_temporal_animation_$id.mp4", fps = 30)

[ Info: Saved animation to c:\Users\emino\OneDrive\Escritorio\Tareas\5to Semestre\Cuantica\Cuantica MP1 QuantumOsc Final\ex2mp\state2_temporal_animation_var.mp4


Plots.AnimatedGif("c:\\Users\\emino\\OneDrive\\Escritorio\\Tareas\\5to Semestre\\Cuantica\\Cuantica MP1 QuantumOsc Final\\ex2mp\\state2_temporal_animation_var.mp4")

In [54]:
#Check probability conservation
p = zeros(Float64, length(t))
for i ∈ 1:length(t)
    p[i] = sum(heatmap_matrix[i,:])*stpx
end

In [55]:
#Plot probability conservation
plot(t, p, title = L"Total probability of $\vert\psi_2\vert^2$ over time $T=\frac{2\pi}{\omega}$ (%$id)", label=false, ylims=(0, 1.2))
savefig("ex2mp\\state2_cop_$id.png")

"c:\\Users\\emino\\OneDrive\\Escritorio\\Tareas\\5to Semestre\\Cuantica\\Cuantica MP1 QuantumOsc Final\\ex2mp\\state2_cop_var.png"